In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import dateutil.relativedelta
import math
import pygsheets
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from adaptation import ExcelReview

In [2]:
token_new_bi = 'HtdQ1pqETq'
token_cosm_bi = 'lpLLZsHACx'

current_date = datetime.today()
c_day = current_date.day
c_month = current_date.month
c_year = current_date.year

yesterday_date = current_date - dateutil.relativedelta.relativedelta(days=1)
y_day = yesterday_date.day
y_month = yesterday_date.month
y_year = yesterday_date.year


seven_month_date = current_date - dateutil.relativedelta.relativedelta(month=7)
s_day = seven_month_date.day
s_month = seven_month_date.month
s_year = seven_month_date.year

In [4]:
cat_str = "Баннер, АВИТО (Ручная подача), Авито (Покупка услуг), Авито (руч. подача), Авито - поднятие объявлений, Авито реклама, Баннер, Баннеры, Баннеры на колышках, Баннеры-г.Челябинск, Баннеры-ручное списание, Брендовая расклейка, Визитки, Визитки г. Пыть-Ях, Визитки-наклейки, Выгрузка за объявления на МЛСН, Выгрузка на сайты, Выгрузка объявлений на сайты (автовыгрузка), Дополнительный статус на Авито, За Баннеры, За баннеры, За визитки, За поднятие объявления на сайте Авито, За рекламу, За рекламу на АВИТО, За услуги расклейщика, Заказ баннера, Заказ баннеров, Заказ баннеров/фолий, Заказ визиток, Изготовление Баннеров-г.Когалым, Изготовление баннеров, Индивидуальная реклама, Использование баннера, Клей для расклейки, Листовки, Листовки , Маркетинг, Монтаж баннера, Наклейка на окно , Наклейка-визитка, Наклейки, Наклейки на авто, Наклейки на автомобиль, Наклейки на стекла, Объявление Авито, Объявление в газеты, Объявление на сайт N1, Объявление на сайт Авито , Объявление на сайты, Объявление на сайты , Объявление на сайты АвитоPro, Объявления в газеты, Объявления на АВИТО, Объявления на сайт 63.ru, Объявления на сайт АВИТО, Объявления на сайт Авито, Объявления на сайт Авито (ручная выгрузка), Объявления на сайт Из рук в руки, Объявления на сайт Сибдом, Объявления на сайт Циан (ручная выгрузка), Объявления на сайт ЯКТ, Объявления на сайте Avito, Объявления на сайты, Оплата размещения на авито, Оплата расклейщикам, Печать расклейки, Платная выгрузка, Применение платного статуса на Авито, Размещения объявлений на сайт авито, Расклейка, Расклейка объявлений , Расклейщик, Реклама, Реклама на АВИТО, Реклама на Авито, Рекламные листовки, Рекламные расходы, Служба сервиса: расклейка, Служба сервиса: фотографирование, Создание баннеров, Создание визиток, Создание наклеек на авто, Списание за баннер, Списание за визитки, Списание за объявления на сайт ЕМЛС, Списание за расклейку, Списание за рекламу на Авито, Списание на рекламу, Списание с бонусного счета (Авито), Строчная реклама, Услуги расклейщика, ЦИАН (ручная подача), Циан (ручная подача), баннеры, визитка в авто, визитки, доп.статусы на Авито, заказ баннера, заказ баннеров, заказ визиток, заказ наклеек, монтаж баннера, наклейка на автомобиль, наклейка на стекло, оплата баннеров, печать визиток, печать расклейки, платный статус, размещение объявлений в газету, расклейка"
cat_list = cat_str.split(', ')

req_zatrati_rekl = {
    "token": token_new_bi,
    "report_id": 393,
    "fields": ["city_name", "fio", "summ", "payment_date"],
    "filter_query":[
        {
            "field": "payment_date",
            "values": [f"{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        },
        {
            "field": "cat_name",
            "values": cat_list
        }
    ],
    "rows": 10000,
    "start": 0
}

def make_table_req_zatrati_rekl(table):
    table['payment_date'] = pd.to_datetime(table['payment_date'])
    table['date'] = table['payment_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    table['summ'] = table['summ'].astype(float)
    end_table = table.pivot_table('summ', ['city_name', 'fio'], 'date', aggfunc="sum")
    end_table = end_table.fillna(0)
    #end_table['summing'] = end_table.sum(axis=1)
    end_table = end_table.reset_index()
    
    date_column = f"{y_year}-{y_month}-{y_day}"
    end_table[date_column] = end_table[date_column].astype(str)
    end_table[date_column] = end_table[date_column].apply(lambda x: x.replace('.', ','))

    return end_table

req_zatrati_rekl = ExcelReview(req_zatrati_rekl)
req_zatrati_rekl.df = make_table_req_zatrati_rekl(req_zatrati_rekl.get_table())

#req_zatrati_rekl.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'a1')
req_zatrati_rekl.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'fio', ['city_name'])

In [ ]:
del req_zatrati_rekl

In [14]:
zayv_type = 'Дистанционная покупка, Новостройки, Покупка вторичная, Покупка гаражи, Покупка загородной, Покупка загородной элитной недвижимости , Покупка коммерческой, Покупка элитная'
zayv_type = zayv_type.split(', ')

zayv_status = 'deposit, free, open'
zayv_status = zayv_status.split(', ')

zayv_reason = ', , Другое, Клиент отказался от фотографирования, На контроле у риэлтора, Нет ответа, Объект на задатке, Объект продан, Объект удален, Отмена, Продажа объекта приостановлена, Риэлтор отказался от фотографирования, Риэлтор самостоятельно сфотографировал объект, Семейные обстоятельства, выбран неверный город, консультация, купили/продали, не актуально (без причин), не доступен, не отвечает, нереальная потребность, нет подходящих объектов, объект находится далеко, отказались из-за комиссии, отложена, думают, передано другому риэлтору, передумали, по давности, сдали/сняли, сняли с другим АН'
zayv_reason = zayv_reason.split(', ')


zayv_na_pokup = {
    'token': token_new_bi,
    'report_id': 242,
    'fields': ['realtor', 'u_city', 'ticket_id'],
    'filter_query': [
        {
            'field': 'type_name',
            'values': zayv_type
        },
        {
            'field': 'status',
            'values': zayv_status
        }, 
        {
            'field': 'reason',
            'values': zayv_reason
        },
        {
            'field': 'realtor_start_date',
            "values": [f"{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_zayv_na_pokup(table):
    table['ticket_id'] = table['ticket_id'].astype(str)
    
    table = table.pivot_table('ticket_id', ['realtor', 'u_city'], aggfunc='count')
    table = table.reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'ticket_id':date_column})
    
    return table


zayv_na_pokup = ExcelReview(zayv_na_pokup)
zayv_na_pokup.df = make_table_zayv_na_pokup(zayv_na_pokup.get_table())
#zayv_na_pokup.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'a1')
zayv_na_pokup.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'realtor', ['u_city'])

In [ ]:
del zayv_na_pokup

In [51]:
city_filter = '--, Abu Dhabi, Alanya, Antalya, Dubai, Northern Cyprus, Pattaya, Phuket, Plzeň, Praha, Romashka, Sharjah, Абакан, Актау, Актобе, Алапаевск, Алексеевка, Алматы, Алушта, Альметьевск, Анапа, Ангарск, Анжеро-Судженск, Арсеньев, Архангельск, Астана, Астрахань, Атырау, Ачинск, Аша, Байконур, Баку, Балашов, Барнаул, Бахчисарай, Белгород, Белогорск, Амурская область, Бердск, Березовский (Кемеровская область), Бийск, Бирюч, Бишкек, Благовещенск, Боровичи, Брянск, Бургас, Валуйки, Вейделевка пгт., Великий Новгород, Верхняя Салда, Владивосток, Волгоград, Волжск, Волжский, Волоконовка пгт., Волхов, Воронеж, Воскресенск, Воткинск, Гатчина, Горно-Алтайск, Горячий Ключ, Грозный, Губкин, Губкинский, Далматово, Дербент, Дзержинск, Дмитров, Домодедово, Другой город, Душанбе, Евпатория, Ейск, Екатеринбург, Ереван, Железногорск, Зеленодольск, Златоуст, Иваново, Ижевск, Ирбит, Иркутск, Ишим, Йошкар-Ола, Казань, Калининград, Калуга, Камышлов, Каспийск, Кемерово, Кириши, Киров, Клин, Ковров, Когалым, Кокшетау, Коломна, Комсомольск-на-Амуре, Короча, Корсаков, Коряжма, Костанай, Кострома, Котлас, Краснодар, Красноярск, Кумертау, Курган, Кызыл, Кыштым, Лангепас, Ленинск-Кузнецкий, Липецк, Луховицы, Лянтор, Майкоп, Махачкала, Междуреченск, Мелеуз, Миасс, Минск, Мичуринск, Можга, Москва, Муравленко, Муром, Набережные Челны, Надым, Назрань, Нальчик, Наро-Фоминск, Невинномысск, Нерчинск, Нефтекамск, Нефтеюганск, Нижневартовск, Нижний Новгород, Нижний Тагил, Новая Уфа, Новодвинск, Новокузнецк, Новороссийск, Новосибирск, Новоуральск, Новый Волгоград, Новый Оскол, Новый Уренгой, Норильск, Ноябрьск, Нягань, Обнинск, Одесса, Озерск, Октябрьский (Башкортостан), Октябрьский (Пермский край), Омск, Омская область, Опочка, Оренбург, Орехово-Зуево, Орёл, Остров, Павлодар, Пенза, Первоуральск, Пермь, Пермь 2, Петрозаводск, Петропавловск, Петропавловск-Камчатский, Печоры, Подольск, Пойковский пгт., Покров, Прокопьевск, Прохладный, Псков, Пыть-Ях, Пятигорск, Ракитное, Раменское, Московская обл., Ростов-на-Дону, Рудный, Рязань, Саки, Салават, Салехард, Самара, Самарканд, Санкт-Петербург, Саранск, Сарапул, Саратов, Сатка, Свободный, Северодвинск, Семей, Симферополь, Смоленск, Сосновоборск, Сочи, Ставрополь, Старый Оскол, Стерлитамак, Судак, Сургут, Сыктывкар, Таганрог, Талица, Тамбов, Тарко-Сале, Ташкент, Тверь, Темрюк, Тобольск, Тольятти, Томск, Троицк, Туапсе, Туймазы, Тула, Туринск, Тюмень, Улан-Удэ, Ульяновск, Уральск, Усинск, Усть-Каменогорск, Уфа, Ухта, Феодосия, Хабаровск, Ханты-Мансийск, Чебаркуль, Челябинск, Череповец, Черкесск, Черногорск, Чита, Шадринск, Шексна, Шымкент, Электросталь, Южно-Сахалинск, Южноуральск, Юрга, Якутск, Ялта, Ялуторовск, Ярославль, Ярцево'
city_filter = city_filter.split(', ')


objects_in_base = {
    'token': token_new_bi, 
    'report_id': 347,
    'fields': ['city', 'fio', 'count_flats', 'count_object', 'count_cottages', 'count_object_contract'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_objects_in_base(table):
    table = table.fillna(0)

    table['count_object'] = table['count_object'].astype(float)
    table['count_flats'] = table['count_flats'].astype(float)
    table['count_object_contract'] = table['count_object_contract'].astype(float)
    table['count_cottages'] = table['count_cottages'].astype(float)

    arrays = [[f'{y_year}-{y_month}-16', 'city', f'{y_year}-{y_month}-16', 'fio', f'{y_year}-{y_month}-16', f'{y_year}-{y_month}-16'], ['count_object', 'city', 'fio', 'count_cottages', 'count_object_contract', 'count_object']]
    table.columns = pd.MultiIndex.from_arrays(arrays)
    
    return table


#objects_in_base = ExcelReview(objects_in_base)
#objects_in_base.df = make_table_objects_in_base(objects_in_base.get_table())
#objects_in_base.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ОбъектыВБазеДень', 'a1')
#objects_in_base.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ОбъектыВБазеДень', 'fio', ['city'])

c:\Users\ws-tmn-an-15\Desktop\Харайкин М.А\Python документы\DayAdaptation\adaptation.py:17: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.table = table
c:\Users\ws-tmn-an-15\Desktop\Харайкин М.А\Python документы\DayAdaptation\adaptation.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.df = pd.DataFrame()
c:\Users\ws-tmn-an-15\Desktop\Харайкин М.А\Python документы\DayAdaptation\adaptation.py:53: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  loaded_table = xlsx_table.merge(self.df, how='outer', on=merge_column)


ValueError: cannot handle a non-unique multi-index!

In [43]:
objects_in_base = ExcelReview(objects_in_base)
objects_in_base.df = objects_in_base.get_table()

In [47]:
arrays = [[f'{y_year}-{y_month}-16', 'city', f'{y_year}-{y_month}-16', 'fio', f'{y_year}-{y_month}-16', f'{y_year}-{y_month}-16'], ['count_object', 'city', 'fio', 'count_cottages', 'count_object_contract', 'count_object']]
objects_in_base.df.columns = pd.MultiIndex.from_product(arrays)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 36 elements

In [37]:
objects_in_base.df

fio            city         1111                                      \
      fio            city count_object                      count_cottages   
0      11          Тюмень           10             Тиунов Антон Васильевич   
1      26  Южно-Сахалинск           13          Паршина Мариана Валерьевна   
2      16          Тюмень           10           Маркин Евгений Викторович   
3     NaN           Dubai          NaN  Mahmoud Al Mahamid Yousef Ghazaleh   
4      72         Душанбе           71         Шомахмадова Робия Фирузовна   
...   ...             ...          ...                                 ...   
5844  NaN          Калуга          NaN       Нарушевич Надежда Анатольевна   
5845  NaN    Екатеринбург          NaN          Приходько Михаил Андреевич   
5846  NaN    Екатеринбург          NaN          Шибанова Анна Владимировна   
5847  NaN          Ижевск          NaN                   Аранкулов Дмитрий   
5848  NaN     Альметьевск          NaN         Аблязов Усман Галиулла углы   

                                         
     count_object_contract count_object  
0                      NaN          NaN  
1                       11            1  
2                      NaN            1  
3                      NaN          NaN  
4                        1          NaN  
...                    ...          ...  
5844                   NaN          NaN  
5845                   NaN          NaN  
5846                   NaN          NaN  
5847                   NaN          NaN  
5848                   NaN          NaN  

[5849 rows x 6 columns]

In [ ]:
podborki = {}

In [ ]:
pokazi_pokupatel = {}

In [ ]:
ipotech_consult = {}

In [ ]:
zayv_ipotek_bank = {}

In [ ]:
#load_by_day - обновляет таблицу на ежедневной основе
#load_full_table - загружает первый день в начале месяца
#для make_sort_table нужны поля из таблицы, которые влияют на merge

In [ ]:
#выкачать всю таблицу, сделать merge со всей таблицей
#т.к. в таблице сгрупированные данные выкачиваться и закачиваться будет не долго